<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/NLP_R%C3%A9sum%C3%A9_d'article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voici l'implémentation que j'ai crée qui divise les articles avant de les résumer.

Mais ça prend trop de temps pour être exécuter sur les 400 articles du train set OBS.

Peut-être l'essayer sur 10 articles ne serait pas une mauvaise idée.

Ici le modèle est déjà préentrainé.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import pandas as pd

# Charger le modèle LED pour résumer des textes longs
model_name = "allenai/led-large-16384"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dossier contenant les fichiers d'articles
dossier_articles = "/kaggle/input/m-2-maliash-resume-darticles-scientifiques/train/OBS/articles_OBS/"

# Fonction pour lire un fichier
def lire_article(chemin_dossier, nom_fichier):
    chemin_complet = os.path.join(chemin_dossier, nom_fichier)
    with open(chemin_complet, 'r', encoding='utf-8') as fichier:
        return fichier.read()

# Fonction pour résumer un texte long
def resumer_texte_long(texte, max_length=512, chunk_size=4096):
    # Diviser le texte en morceaux
    morceaux = [texte[i:i + chunk_size] for i in range(0, len(texte), chunk_size)]
    resumes = []
    for morceau in morceaux:
        # Générer un résumé pour chaque morceau
        inputs = tokenizer(morceau, return_tensors="pt", truncation=True, max_length=chunk_size)
        outputs = model.generate(inputs["input_ids"], max_length=max_length, min_length=50, length_penalty=2.0)
        resumes.append(tokenizer.decode(outputs[0], skip_special_tokens=True))

    # Combiner les résumés partiels en un seul texte
    texte_final = " ".join(resumes)
    return texte_final

# Lister tous les fichiers d'articles
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Liste pour stocker les résumés générés
resumes_generes = []

# Traiter chaque fichier d'article
for article_fichier in fichiers_articles:
    try:
        # Lire le contenu de l'article
        contenu_article = lire_article(dossier_articles, article_fichier)

        # Résumer l'article
        resume = resumer_texte_long(contenu_article, max_length=512, chunk_size=4096)

        # Ajouter le résumé et l'identifiant à la liste
        resumes_generes.append({
            "fichier": article_fichier,
            "resume": resume
        })

        # Afficher un message de progression
        print(f"Résumé généré pour : {article_fichier}")

    except Exception as e:
        print(f"Erreur pour l'article {article_fichier}: {e}")

# Sauvegarder les résumés dans un fichier CSV
df_resumes = pd.DataFrame(resumes_generes)
df_resumes.to_csv("resumes_articles.csv", index=False, encoding="utf-8")
print("Tous les résumés ont été sauvegardés dans 'resumes_articles.csv'")
